In [1]:

import torch
import torch.nn as nn

/home/akil/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, output_channels, kernel, stride, padding, debug=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, output_channels, kernel, stride, padding)
        self.batchNorm = nn.BatchNorm2d(output_channels)
        self.ReLU = nn.ReLU()
        self.debug = debug
        self.kernel = kernel
    
    def forward(self, x):
        if self.debug:
            print(f'Shape Before Convolution: {x.shape}')
        
        x = self.conv(x)
        if self.debug:
            print(f'Shape After Convolution with {self.kernel} x {self.kernel}: {x.shape}')
        
        x = self.batchNorm(x)
        
        x = self.ReLU(x)
        
        return x

In [5]:
class InceptionBlock(nn.Module):
    def __init__(self, input_channels, output_channel_list):
        super(InceptionBlock, self).__init__()
        self.path_1 = ConvBlock(input_channels, output_channel_list[0],1, 1, 0, True)
        
        self.path_2 = nn.Sequential(
            ConvBlock(input_channels, output_channel_list[1], 1, 1, 0, True),
            ConvBlock(output_channel_list[1], output_channel_list[2], 3, 1, 1, True)
        )
        
        self.path_3 = nn.Sequential(
            ConvBlock(input_channels, output_channel_list[3], 1, 1, 0, True),
            ConvBlock(output_channel_list[3], output_channel_list[4], 5, 1, 2, True)
        )
        
        self.path_4 = nn.Sequential(
            nn.MaxPool2d(3, 1, 1),
            ConvBlock(input_channels, output_channel_list[5], 1, 1, 0, True)
        )
        
    def forward(self, x):
        a = self.path_1(x)
        b = self.path_2(x) 
        c = self.path_3(x) 
        d = self.path_4(x)
        
        print(a.shape)
        print(b.shape)
        print(c.shape)
        print(d.shape)
        
        concatenated_tensor = torch.cat((a, b, c, d), dim=1)
        print(f'Shape After Concatenation {concatenated_tensor.shape}')
        
        return concatenated_tensor

In [6]:
InceptionBlock(3, [64, 96, 128, 16, 32, 32])(torch.randn(32, 3, 224, 224)).to(device).shape

Shape Before Convolution: torch.Size([32, 3, 224, 224])
Shape After Convolution with 1 x 1: torch.Size([32, 64, 224, 224])
Shape Before Convolution: torch.Size([32, 3, 224, 224])
Shape After Convolution with 1 x 1: torch.Size([32, 96, 224, 224])
Shape Before Convolution: torch.Size([32, 96, 224, 224])
Shape After Convolution with 3 x 3: torch.Size([32, 128, 224, 224])
Shape Before Convolution: torch.Size([32, 3, 224, 224])
Shape After Convolution with 1 x 1: torch.Size([32, 16, 224, 224])
Shape Before Convolution: torch.Size([32, 16, 224, 224])
Shape After Convolution with 5 x 5: torch.Size([32, 32, 224, 224])
Shape Before Convolution: torch.Size([32, 3, 224, 224])
Shape After Convolution with 1 x 1: torch.Size([32, 32, 224, 224])
torch.Size([32, 64, 224, 224])
torch.Size([32, 128, 224, 224])
torch.Size([32, 32, 224, 224])
torch.Size([32, 32, 224, 224])
Shape After Concatenation torch.Size([32, 256, 224, 224])


torch.Size([32, 256, 224, 224])

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(self, input_channels, output_channel_list):
        pass

In [ ]:
inception_block_out_channels = [
    out_1_1,
    out_red_3_3,
    out_3_3,
    out_red_5_5,
    out_5_5,
    out_pool_proj
]

In [17]:
x = torch.FloatTensor(28,28, 64)
y = torch.FloatTensor(28,28, 128)

torch.cat((x, y) , dim=-1).shape


torch.Size([28, 28, 192])